# Imports

In [3]:
import os
import requests
from IPython.display import display, HTML
from tabulate import tabulate

## Helper functions

In [4]:

def fetch_all_data(url):
    """Fetch all data from the given URL."""
    print("retrieving data from", url)
    counter = 0 
    documents = []
    while url and counter < 1000:
        response = requests.get(url)
        data = response.json()
        documents.extend(data.get('value', []))
        url = data.get('@odata.nextLink', None)  # Get the next link if available
        counter += 1
    return documents

def truncate_string(s, max_length):
    """Truncate string to the max_length with ellipsis if needed."""
    return s if len(s) <= max_length else s[:max_length - 3] + '...'

## Basic OData filtering system
Info hierover kan je vinden op de website van de [Tweede Kamer OData](https://opendata.tweedekamer.nl/documentatie/odata-api). Alternatief kan je op de [Officiele documentatie van OData kijken](https://www.odata.org/documentation/)



In [5]:
# Function to construct URL with filters
def construct_url(base_url, filters=None, select=None, expand=None, skip=None, top=None, orderby=None, count=False):
    query_params = []

    if filters:
        query_params.append(f"$filter={filters}")
    if select:
        query_params.append(f"$select={select}")
    if expand:
        query_params.append(f"$expand={expand}")
    if skip:
        query_params.append(f"$skip={skip}")
    if top:
        query_params.append(f"$top={top}")
    if orderby:
        query_params.append(f"$orderby={orderby}")
    if count:
        query_params.append(f"$count=true")

    if query_params:
        return f"{base_url}?" + "&".join(query_params)
    else:
        return base_url

In [6]:
base_url = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
documentation_url = "https://opendata.tweedekamer.nl/documentatie/"


# Filters en params
filters = None # Kan bijvoorbeeld zijn: 'Kamer eq 'tweede kamer' and year(datum) eq 2024'
select = None # Kan bijvoorbeeld zijn: 'id, naam, partij'
expand = None # Bedoeld om child-data op te vragen, 
skip = None # Skip aantal records
top = None # Top aantal records
orderby = None # Orderby asc, desc
count = False # Count aantal records

# Maak url
url = construct_url(base_url, filters, select, expand, skip, top, orderby, count)
print(url)

https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/


## Alle mogelijke endpoints binnen OData

Meer info kan je vinden in het [odata informatiemodel van de tweede kamer](https://opendata.tweedekamer.nl/documentatie/informatiemodel)

In [7]:
# Get data
endpoints = requests.get(url)

# Process to HTML
data = []
for endpoint in endpoints.json()['value']:
    endpoint_name = endpoint['name']
    endpoint_url = base_url + endpoint['url']
    doc_url = documentation_url + endpoint['url']
    # Convert to hyperlinks
    doc_link = f'<a href="{doc_url}" target="_blank">{endpoint_name}</a>'
    endpoint_link = f'<a href="{endpoint_url}" target="_blank">{endpoint_url}</a>'
    data.append([doc_link, endpoint_link])

# Using tabulate to create an HTML table
table = tabulate(data, headers=['Endpoint naam + doc URL', 'Endpoint URL', 'Documentatie URL'], tablefmt='unsafehtml')

# Displaying the table
display(HTML(table))

Endpoint naam + doc URL,Endpoint URL
Persoon,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Persoon
PersoonContactinformatie,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonContactinformatie
PersoonGeschenk,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonGeschenk
PersoonLoopbaan,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonLoopbaan
PersoonNevenfunctie,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonNevenfunctie
PersoonNevenfunctieInkomsten,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonNevenfunctieInkomsten
PersoonOnderwijs,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonOnderwijs
PersoonReis,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/PersoonReis
Commissie,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Commissie
CommissieContactinformatie,https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/CommissieContactinformatie


## Alle verslagen ophalen van vergaderingen
Volgens het OData model heeft een vergadering 1 of meerdere verslagen. Met de OData filter parameters kunnen wij specificeren welke vergaderingen wij willen, en dat we alle verslagen die daarbij horen ook willen .meenemen. Dat kunnen we doen d.m.v. de $expand filter. Deze zit ingebouwd in OData, en is voor ons relatief makkelijk te gebruiken. We hoeven enkel aan te geven welke relatie we mee willen nemen. In dit geval, Verslag.

### Alle vergaderingen ophalen

In [51]:
# Set up URL to retrieve data
url = "https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering"
filter = "Kamer eq 'tweede kamer' and year(datum) eq 2024" 
orderby = "Datum desc" # Orderby asc, desc
expand = "Verslag"

combined_url = construct_url(url, filters=filter, orderby=orderby, expand=expand)
vergadering_data = fetch_all_data(combined_url)

# Define placeholder for data
data = []

# Loop over vergaderingen en truncate titel
for vergadering in vergadering_data:
    titel = truncate_string(vergadering['Titel'], 50)
    data.append([titel, vergadering['Datum'], vergadering['Zaal'], vergadering.get('Verslag')[-1].get('Soort')])
    
# Display data using tabulate
print(tabulate(data, headers=['Titel', 'Datum', 'Zaal', 'Verslag soort']))

retrieving data from https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering?$filter=Kamer eq 'tweede kamer' and year(datum) eq 2024&$expand=Verslag&$orderby=Datum desc
Titel                                               Datum                      Zaal                          Verslag soort
--------------------------------------------------  -------------------------  ----------------------------  ----------------
92e vergadering, donderdag 4 juli 2024              2024-07-04T00:00:00+02:00  Plenaire zaal                 Tussenpublicatie
Mestbeleid                                          2024-07-04T00:00:00+02:00  Thorbeckezaal                 Tussenpublicatie
91e vergadering, woensdag 3 juli 2024               2024-07-03T00:00:00+02:00  Plenaire zaal                 Tussenpublicatie
NAVO                                                2024-07-03T00:00:00+02:00  Thorbeckezaal                 Tussenpublicatie
90e vergadering, donderdag 27 juni 2024             2024-06-27T00:00:

## Alle actieve sprekers uit de tweede kamer ophalen

In [50]:
base_url = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
documentation_url = "https://opendata.tweedekamer.nl/documentatie/"

# All people who weren't removed and are Tweede Kamerlid
filters = "Verwijderd eq false and Functie eq 'Tweede Kamerlid'"

# Expand the FractieZetelPersoon, which is linked to the Persoon. From there, retrieve the FractieZetel, and Fractie, and select the name of the Fractie
expand = "FractieZetelPersoon($expand=FractieZetel($expand=Fractie($select=NaamNL)))"

# Create url
url = construct_url(base_url + 'Persoon', filters=filters, expand=expand)
url

"https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Persoon?$filter=Verwijderd eq false and Functie eq 'Tweede Kamerlid'&$expand=FractieZetelPersoon($expand=FractieZetel($expand=Fractie($select=NaamNL)))"

In [47]:
politici = fetch_all_data(url)

data = []

for politicus in politici:
    # Not using .get() for the sake of readability
    naam = politicus['Voornamen'] + ' ' + politicus['Achternaam']
    kamer = politicus['Functie']
    partij = politicus['FractieZetelPersoon'][0]['FractieZetel']['Fractie']['NaamNL']
    data.append([naam,partij, kamer])
    
print(tabulate(data, headers=['Naam', 'Partij', 'Kamer']))

retrieving data from https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Persoon?$filter=Verwijderd eq false and Functie eq 'Tweede Kamerlid'&$expand=FractieZetelPersoon($expand=FractieZetel($expand=Fractie($select=NaamNL)))
Naam                                           Partij                                   Kamer
---------------------------------------------  ---------------------------------------  ---------------
Ismail Abassi                                  DENK                                     Tweede Kamerlid
Eric René Franciscus Esser                     Partij voor de Vrijheid                  Tweede Kamerlid
Inge Dijk                                      Christen-Democratisch Appèl              Tweede Kamerlid
Johannes Alexander Vijlbrief                   Democraten 66                            Tweede Kamerlid
Ulysse Ellian                                  Volkspartij voor Vrijheid en Democratie  Tweede Kamerlid
Mohamed Mohandis                               Partij va